# LangGraph: Сложная State Machine с RAG

**Цель:** граф с 6 узлами, двумя условными рёбрами и двумя циклами — видна полная трассировка обхода.

**Граф:**
```
                    ┌─────────────────────────────────────┐
                    │  router_ctx: контекст короткий?      │
                    └─────────────────────────────────────┘
START → retrieve ──┬→ enrich → retrieve (цикл обогащения)
                   └→ generate
                         │
                    ┌────┴──── router_quality: ответ короткий?
                    ↓         ↓
                 critique   END
                    │
                  refine → generate (цикл улучшения)
```

---

### 🎯 Якорь
Два цикла, два роутера: `retrieve↔enrich` (контекст) и `generate↔critique↔refine` (качество ответа).

In [ ]:
# ← 1. Установка (Colab: Runtime → GPU)
!pip install -q langgraph langchain langchain-core langchain-community
!pip install -q transformers accelerate torch

> ⚠️ **Если ошибка `torchvision::nms does not exist`:** используй `model.generate()` напрямую вместо pipeline — см. [ie_extraction_hw](09_hw_ie_extraction) как обойти импорт pipeline.

## 1. Загрузка малой LLM (TinyLlama 1.1B)

TinyLlama помещается в Colab T4 (~2–4 GB VRAM). Без 4-bit — быстрее для демо.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # открытая, ~2.2 GB

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe)
print("LLM загружена, device:", next(model.parameters()).device)

## 2. State и узлы графа

- **State:** messages, context, enrich_count, critique_count, feedback
- **retrieve:** подтягивает контекст (длина зависит от enrich_count)
- **enrich:** увеличивает enrich_count, цикл в retrieve
- **generate:** LLM по контексту и вопросу (или с feedback из critique)
- **critique:** анализирует ответ, ставит feedback
- **refine:** перезапускает generate с учётом feedback

In [ ]:
from typing import TypedDict
from langchain_core.messages import HumanMessage, AIMessage

# ← 1. Состояние графа
class State(TypedDict):
    messages: list
    context: str
    enrich_count: int
    critique_count: int
    feedback: str

# Демо: "документы" наращиваются при enrich (симулируем многораундовый retrieve)
CHUNK = "RAG — Retrieval-Augmented Generation. Поиск документов + LLM по контексту."
FULL_DOCS = CHUNK + " " + "Система подкладывает релевантные документы в промпт. LLM отвечает на основе контекста."

def retrieve(state: State) -> dict:
    """Узел 1: подтягиваем контекст. Длина растёт с enrich_count."""
    ec = state.get("enrich_count", 0)
    if TRACE:
        src = "START" if ec == 0 else "enrich"
        print(f"\n  [УЗЕЛ] retrieve ← по ребру {src}→retrieve")
    ctx = CHUNK if ec == 0 else FULL_DOCS + " [доп. раунд " + str(ec) + "]"
    return {"context": ctx}

def enrich(state: State) -> dict:
    """Узел 2: обогащаем — увеличиваем счётчик, пойдём снова в retrieve."""
    ec = state.get("enrich_count", 0) + 1
    if TRACE:
        print(f"\n  [УЗЕЛ] enrich ← по ребру router_ctx(enrich)→enrich")
    return {"enrich_count": ec}

def generate(state: State) -> dict:
    """Узел 3: генерируем ответ. Если есть feedback — учитываем при refine."""
    src = "refine→generate" if state.get("feedback") else "retrieve→generate"
    if TRACE:
        print(f"\n  [УЗЕЛ] generate ← по ребру {src} (жёсткое)")
    question = state["messages"][-1].content if state["messages"] else ""
    fb = state.get("feedback", "")
    extra = f"\nУчти замечание: {fb}" if fb else ""
    prompt = f"""Контекст: {state['context']}\nВопрос: {question}{extra}\nКраткий ответ:"""
    response = llm.invoke(prompt)
    return {"messages": state["messages"] + [AIMessage(content=response)], "feedback": ""}

def critique(state: State) -> dict:
    """Узел 4: критикуем короткий ответ, задаём feedback для refine."""
    if TRACE:
        print(f"\n  [УЗЕЛ] critique ← по ребру router_quality(critique)→critique")
    last = state["messages"][-1]
    text = last.content if hasattr(last, "content") else str(last)
    return {
        "feedback": "Дай более развёрнутый ответ, минимум 2-3 предложения.",
        "critique_count": state.get("critique_count", 0) + 1,
    }

def refine(state: State) -> dict:
    """Узел 5: передаём управление в generate с feedback (состояние уже обновлён critique)."""
    if TRACE:
        print(f"\n  [УЗЕЛ] refine ← по ребру critique→refine (жёсткое)")
    return {}  # generate возьмёт feedback из state

# Включить трассировку обхода графа (False = без логов)
TRACE = True

## 3. Роутеры: условные рёбра

- **router_ctx** (после retrieve): контекст короткий (< 100 символов) и enrich_count < 2 → enrich, иначе → generate
- **router_quality** (после generate): ответ короткий (< 30 символов) и critique_count < 2 → critique, иначе → END

In [ ]:
def router_ctx(state: State) -> str:
    """Роутер 1: контекст короткий? → enrich (цикл), иначе → generate."""
    ctx = state.get("context", "")
    ec = state.get("enrich_count", 0)
    short_ctx = len(ctx.strip()) < 100
    if short_ctx and ec < 2:
        if TRACE:
            print(f"  [ВЕТВЛЕНИЕ] router_ctx: context={len(ctx)} символов, enrich_count={ec} → enrich")
        return "enrich"
    if TRACE:
        print(f"  [ВЕТВЛЕНИЕ] router_ctx: context={len(ctx)} символов → generate")
    return "generate"

def router_quality(state: State) -> str:
    """Роутер 2: ответ короткий? → critique (цикл refine), иначе → end."""
    last = state["messages"][-1] if state.get("messages") else None
    text = (last.content if hasattr(last, "content") else str(last)) if last else ""
    cc = state.get("critique_count", 0)
    short = len(text.strip()) < 30
    if short and cc < 2:
        if TRACE:
            print(f"  [ВЕТВЛЕНИЕ] router_quality: ответ={len(text.strip())} символов, critique_count={cc} → critique")
        return "critique"
    if TRACE:
        print(f"  [ВЕТВЛЕНИЕ] router_quality: ответ={len(text.strip())} символов → end")
    return "end"

## 4. Сборка графа: узлы + рёбра

- Жёсткие: `enrich→retrieve`, `critique→refine`, `refine→generate`
- Условные: `retrieve→[router_ctx]→enrich|generate`, `generate→[router_quality]→critique|END`

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(State)

# Узлы
workflow.add_node("retrieve", retrieve)
workflow.add_node("enrich", enrich)
workflow.add_node("generate", generate)
workflow.add_node("critique", critique)
workflow.add_node("refine", refine)

# Старт
workflow.set_entry_point("retrieve")

# Условное ребро 1: retrieve → [router_ctx] → enrich | generate
workflow.add_conditional_edges("retrieve", router_ctx, {"enrich": "enrich", "generate": "generate"})

# Жёсткое: enrich → retrieve (цикл обогащения контекста)
workflow.add_edge("enrich", "retrieve")

# Условное ребро 2: generate → [router_quality] → critique | END
workflow.add_conditional_edges("generate", router_quality, {"critique": "critique", "end": END})

# Жёсткие: critique → refine → generate (цикл улучшения ответа)
workflow.add_edge("critique", "refine")
workflow.add_edge("refine", "generate")

app = workflow.compile()
print("Граф скомпилирован: 6 узлов, 2 условных рёбра, 2 цикла (enrich↔retrieve, critique→refine→generate)")

## 5. Трассировка: как граф ходит по рёбрам

При `invoke()` будет видно:
- **УЗЕЛ** — вход в узел и по какому ребру пришли
- **ВЕТВЛЕНИЕ** — решение роутеров: `router_ctx` (enrich|generate), `router_quality` (critique|end)

## 6. Запуск

In [ ]:
print("=== Трассировка обхода графа ===")
result = app.invoke(
    {
        "messages": [HumanMessage(content="Что такое RAG?")],
        "context": "",
        "enrich_count": 0,
        "critique_count": 0,
        "feedback": "",
    },
    config={"recursion_limit": 15},  # защита от бесконечных циклов
)

print("\n=== Результат ===")
for i, m in enumerate(result["messages"]):
    role = "User" if isinstance(m, HumanMessage) else "Assistant"
    print(f"[{role}] {m.content[:200]}{'...' if len(str(m.content)) > 200 else ''}")

print("\nenrich_count:", result.get("enrich_count", 0), "| critique_count:", result.get("critique_count", 0))

## Схема графа

```
     ┌──────────┐
     │  retrieve │  ← entry
     └─────┬─────┘
           │ add_conditional_edges (router_ctx)
     ┌─────┴─────┐
     ↓           ↓
 enrich      generate
     │           │
     │ add_edge  │ add_conditional_edges (router_quality)
     ↓           ├──────────┐
 retrieve      critique   end → END
 (цикл)           │
                  │ add_edge
                  ↓
               refine
                  │ add_edge
                  ↓
               generate
              (цикл)
```